In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F



n_batch = 10    # batch size
n_channel = 3
momentum = 0.1

bn = nn.BatchNorm2d(n_channel, momentum = momentum, affine=False)  ## learnable parameters not used by setting affine = "False"

print("learning mean is:", bn.running_mean)   # 初始化值 initial value
print("learning var is:",  bn.running_var)    # initial value

input = torch.randn(n_batch, n_channel, 2, 2)   # (batch size, channel, height, width)  --- a minibatch declaration
output = bn(input)

print("learning mean is:", bn.running_mean)
print("learning var is:",  bn.running_var)

bn.eval()  # without bn.eval(), the mean & var will be updated again
input_new = torch.randn(n_batch, n_channel, 2, 2)
output = bn(input_new)

print("learning mean is:", bn.running_mean)
print("learning var is:",  bn.running_var)

learning mean is: tensor([0., 0., 0.])
learning var is: tensor([1., 1., 1.])
learning mean is: tensor([-0.0297,  0.0171, -0.0101])
learning var is: tensor([0.9959, 0.9776, 1.0074])
learning mean is: tensor([-0.0297,  0.0171, -0.0101])
learning var is: tensor([0.9959, 0.9776, 1.0074])


In [2]:
mean = torch.zeros(n_channel)

for ch in range(n_channel):
    for batch in range(n_batch):
        mean[ch] += (input[batch,ch,0,0] + input[batch,ch,0,1] + input[batch,ch,1,0] + input[batch,ch,1,1])
    print("manually calculated mean:", mean[ch]/(n_batch*2*2)*momentum)

manually calculated mean: tensor(-0.0297)
manually calculated mean: tensor(0.0171)
manually calculated mean: tensor(-0.0101)


In [7]:
var = torch.zeros(n_channel)
mean2 = torch.zeros(n_channel)
for ch in range(n_channel):
    for batch in range(n_batch):
        mean2[ch] += (input[batch,ch,0,0] + input[batch,ch,0,1] + input[batch,ch,1,0] + input[batch,ch,1,1])
    mean2[ch] = mean2[ch]/(n_batch*2*2)
    for batch in range(n_batch):
        var[ch] += ( (input[batch,ch,0,0] - mean2[ch] )**2 + (input[batch,ch,0,1] - mean2[ch] )**2 + (input[batch,ch,1,0] - mean2[ch] )**2 + (input[batch,ch,1,1] - mean2[ch] )**2 )
    var[ch] = var[ch]/(n_batch*2*2)*momentum + (1-momentum)
    print("manually calculated var:", var[ch], ch)

manually calculated var: tensor(0.9935) 0
manually calculated var: tensor(0.9757) 1
manually calculated var: tensor(1.0047) 2
